In [1]:
!python -m pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.5/553.5 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.89.1
    Uninstalling fastapi-0.89.1:
      Successfully uninstalled fastapi-0.89.1


## Setup

In [2]:
import torchvision
from torchmetrics import F1Score
import torch
import lightning.pytorch as pl
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import timm
import PIL
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [3]:
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [4]:
# CONSTANTS

DATA_DIR = '/kaggle/input/vk-made-sports-image-classification/'
TEST_DIR = DATA_DIR + 'test/'
TRAIN_DIR = DATA_DIR + 'train/'

# Data load & split

In [5]:
data = pd.read_csv(DATA_DIR + 'train.csv')
delayed = pd.read_csv(DATA_DIR + 'test.csv')
data.image_id = TRAIN_DIR + data.image_id
delayed.image_id =  TEST_DIR + delayed.image_id
delayed['label'] = 'TO_BE_PREDICTED'
LABEL_MAPPING = {label: idx for idx, label in enumerate(data.label.unique())}
NUM_CLASSES = len(LABEL_MAPPING)

In [6]:
train_df, val_df = train_test_split(data, test_size=0.2, stratify=data.label)

### Torch Dataset

In [7]:
class ImageDataset(Dataset):
    
    label_mapping = LABEL_MAPPING

    def __init__(self, dataframe, transform = None):
        super(Dataset, self).__init__()
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path, label = self.dataframe.iloc[idx]
        image = Image.open(image_path).convert('RGB')
        label_id = LABEL_MAPPING[label] if label in LABEL_MAPPING else -1
        if self.transform:
            image = self.transform(image)

        return image, label_id

## Preprocessing and augmentations

In [8]:
import torchvision.transforms.functional as Func

class SquarePad:
    def __call__(self, image):
        w, h = image.shape[-2:]
        max_wh = np.max([w, h])
        hp = int((max_wh - w) / 2)
        vp = int((max_wh - h) / 2)
        padding = (hp, vp, hp, vp)
        return Func.pad(image, padding, 0, 'constant')
    
    
resize_normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        SquarePad(),
        transforms.Resize((224,224), interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
        transforms.Normalize(
            mean = torch.tensor([0.4850, 0.4560, 0.4060]),
            std  = torch.tensor([0.2290, 0.2240, 0.2250])
        )
    ]
)

augs_transform =[
        transforms.RandomCrop((224,224)),
        transforms.RandomGrayscale(p = 0.2),
        transforms.RandomAffine((-10,10)),
        transforms.RandomHorizontalFlip(1),
        transforms.RandomPerspective()
    ]

    
augment_resize_normalize = transforms.Compose(
    [
        transforms.Resize(300),
        transforms.RandomApply(augs_transform),
        resize_normalize,
    ]
)

## Dataloaders

In [9]:
full_dataset = ImageDataset(data,augment_resize_normalize)
train_dataset = ImageDataset(train_df,augment_resize_normalize)
val_dataset =  ImageDataset(val_df,resize_normalize)
delayed_dataset =  ImageDataset(delayed,resize_normalize)

In [10]:
TRAIN_BATCH_SIZE = 64
INFERENCE_BATCH_SIZE = 256
PIN_MEMORY = True
NUM_WORKERS = 2

full_dataloader = DataLoader(
    full_dataset, 
    batch_size = TRAIN_BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = PIN_MEMORY
)

train_dataloader = DataLoader(
    train_dataset, 
    batch_size = TRAIN_BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = PIN_MEMORY
)

val_dataloader = DataLoader(
    val_dataset, 
    batch_size = INFERENCE_BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = PIN_MEMORY
)

delayed_dataloader = DataLoader(
    delayed_dataset, 
    batch_size = INFERENCE_BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = PIN_MEMORY
)

## Model

In [11]:
class SEResNeXTClassifier(pl.LightningModule):
    def __init__(self, model_name='seresnext50_32x4d', num_classes=30, freeze_backbone = True):
        super().__init__()
        self.num_classes = num_classes
        self.model = timm.create_model(model_name, pretrained=True)
        if freeze_backbone:
            for param in self.model.parameters():
                param.requires_grad = False
        self.model.fc = torch.nn.Linear(in_features=2048,out_features=num_classes)
        
        #self.loss = torch.nn.NLLLoss()
        self.loss = torch.nn.CrossEntropyLoss()
        self.metric = F1Score(
            task = 'multiclass',
            num_classes= self.num_classes,
            average='micro',
        )

    def forward(self, x):
        logits = self.model.forward(x)
        probs = F.softmax(logits)
        return probs

    def training_step(self, batch, batch_idx):
        x, y = batch
        probs = self.model.forward(x)
        loss = self.loss(probs, y)
        self.log("train_loss", loss, on_step=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        probs = self.model.forward(x)
        loss = self.loss(probs, y)
        pred_labels = torch.argmax(probs,dim = 1)
        val_score = self.metric(pred_labels, y)
        
        self.log("val_score", val_score, on_epoch=True,prog_bar=True,logger=True)
        self.log("val_loss", loss, on_epoch=True,prog_bar=True,logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        probs = self.model.forward(x)
        loss = self.loss(probs, y)
        pred_labels = torch.argmax(probs,dim = 1)
        test_score = self.metric(pred_labels, y)
        
        self.log("test_score", test_score, on_epoch=True, logger=True, prog_bar=True)
        self.log("val_loss", loss, on_epoch=True,logger=True, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        #optimizer = torch.optim.SGD(self.parameters(), lr= 0.001, momentum = 0.95, nesterov = True,weight_decay=0.0001)
        #return torch.optim.Adam(self.parameters(), lr=0.001)
    
        optimizer = torch.optim.Adam(self.parameters())
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3, verbose = True),
                "interval": "epoch",
                "frequency": 1,
                "monitor": "val_score",
                'reduce_on_plateau': True,
                "strict": True,
                "name": "Reduce LR on plateau",
            },
        }
    

## Training

In [12]:
model = SEResNeXTClassifier(model_name='seresnext101_32x4d', freeze_backbone=False)

checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_score")

trainer = pl.Trainer(
    max_epochs=20,
    accelerator='gpu', 
    devices=1,
    callbacks=[
        #pl.callbacks.early_stopping.EarlyStopping(monitor="val_score", patience=10),
        pl.callbacks.LearningRateMonitor(logging_interval="step"),
        checkpoint_callback
    ])

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, full_dataloader, val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name   | Type              | Params
---------------------------------------------
0 | model  | ResNet            | 47.0 M
1 | loss   | CrossEntropyLoss  | 0     
2 | metric | MulticlassF1Score | 0     
---------------------------------------------
47.0 M    Trainable params
0         Non-trainable params
47.0 M    Total params
187.872   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Epoch 00017: reducing learning rate of group 0 to 1.0000e-07.


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.


In [14]:
trainer.test(model,val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_score         │    0.8862810134887695     │
│         val_loss          │    0.41886526346206665    │
└───────────────────────────┴───────────────────────────┘

[{'test_score': 0.8862810134887695, 'val_loss': 0.41886526346206665}]

# Submission

In [15]:
from tqdm import tqdm
model.to('cuda');
model.eval()
predictions = []
with torch.no_grad():
    for image,label in tqdm(delayed_dataloader):
        prediction = model.forward(image.to('cuda')).argmax(dim = 1)
        predictions.append(prediction)

100%|██████████| 76/76 [02:49<00:00,  2.23s/it]


In [16]:
preds = torch.cat(predictions, dim = -1)
preds = preds.cpu().numpy()

In [17]:
REVERSE_REPLACEMENT = {v:k for k,v in LABEL_MAPPING.items()}

In [18]:
submission_df = pd.read_csv(DATA_DIR + 'test.csv')
submission_df['label'] = [REVERSE_REPLACEMENT[x] for x in preds]

In [19]:
submission_df

,image_id,label
0,00fd3c23-193c-480a-aef9-bb438d50d79e.jpeg,ski_race
1,ef5473b9-a558-4f38-acd0-be4ecfde5a23.jpeg,basketball
2,ba00f9b4-7cbf-4110-91ea-ed41f5cb4ee4.jpeg,alpinism
3,f1bdf877-4379-4e00-b5b8-1e90bdcbda76.jpeg,tennis
4,4c96ff83-07e3-45a4-934f-a92ebf49b299.jpeg,fencing
...,...,...
19441,01ec2a50-db12-4b98-94da-1dd9372650cf.jpeg,skating
19442,1596a0c0-5772-4309-8781-47e15dcdd5d5.jpeg,boxing
19443,65610b2c-c137-468a-85a6-a3ec2ed9e87f.jpeg,basketball
19444,5630e635-fce1-4a33-8748-b7416d16fffc.jpeg,volleyball


In [20]:
submission_df.to_csv('submission_.csv', index = False)

In [21]:
%load_ext tensorboard

In [22]:

%tensorboard --logdir /kaggle/working/lightning_logs/

In [23]:
!kill 3446

/bin/bash: line 0: kill: (3446) - No such process
